In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
manuscript_min_tf = 10
manuscript_max_tf = 0.9
state_path = 's3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev'

In [2]:
import os
from functools import partial
from itertools import groupby
from typing import List, Tuple, TypeVar

import joblib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import to_gbq
from data_science_pipeline.utils.io import load_object_from
from data_science_pipeline.utils.misc import identity_fn
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query
)
from data_science_pipeline.peerscout.models import (
    WeightedKeywordModel
)

In [3]:
model_path = os.path.join(state_path, 'senior_editor_model.joblib')
recommendation_output_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_recommendation'
)

In [4]:
print('loading model from:', model_path)
model_dict = load_object_from(model_path)
model_dict.keys()

loading model from: s3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev/senior_editor_model.joblib


dict_keys(['editor_tf_idf_vectorizer', 'doc_length_weighted_editor_tf_idf', 'editor_names', 'editor_person_ids'])

In [5]:
editor_tf_idf_vectorizer = model_dict['editor_tf_idf_vectorizer']
editor_tf_idf_vectorizer

TfidfVectorizer(lowercase=False, smooth_idf=False, token_pattern=None,
                tokenizer=<function identity_fn at 0x7fbf9d054dd0>)

In [6]:
doc_length_weighted_editor_tf_idf = model_dict['doc_length_weighted_editor_tf_idf']
doc_length_weighted_editor_tf_idf

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00139157, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00150745, 0.00109185, 0.        , ..., 0.        , 0.00292359,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.01753493,
         0.        ],
        [0.        , 0.0010018 , 0.        , ..., 0.00068432, 0.00076642,
         0.        ],
        [0.00065092, 0.00141439, 0.        , ..., 0.        , 0.00216414,
         0.        ]])

In [7]:
editor_names = model_dict['editor_names']
editor_names

0           Aleksandra Walczak
1                  Andrew King
2               Anna Akhmanova
3     Barbara Shinn-Cunningham
4                Carla Rothlin
                ...           
60                 Tamar Makin
61             Timothy Behrens
62              Utpal Banerjee
63              Vivek Malhotra
64               Wendy Garrett
Name: name, Length: 65, dtype: object

In [8]:
editor_person_ids = model_dict['editor_person_ids']
editor_person_ids

0     50904
1     14601
2      8518
3     19576
4     44396
      ...  
60    18331
61     1044
62     1042
63     1133
64    28627
Name: person_id, Length: 65, dtype: object

In [9]:
editor_person_id_by_name_map = dict(zip(editor_names, editor_person_ids))
editor_person_id_by_name_map

{'Aleksandra Walczak': '50904',
 'Andrew King': '14601',
 'Anna Akhmanova': '8518',
 'Barbara Shinn-Cunningham': '19576',
 'Carla Rothlin': '44396',
 'Catherine Dulac': '1014',
 'Chris Baker': '28129',
 'Christian Büchel': '16197',
 'Christian Hardtke': '1102',
 'Christian Rutz': '15332',
 'Clifford Rosen': '48282',
 'Cynthia Wolberger': '5005',
 'David Ron': '1174',
 'Detlef Weigel': '1030',
 'Didier Stainier': '7189',
 'Diethard Tautz': '1191',
 'Dominique Soldati-Favre': '41647',
 'Eduardo Franco': '1086',
 'Edward Morrisey': '55645',
 'Floris de Lange': '28130',
 'Gary Westbrook': '1202',
 'George Perry': '42011',
 'Gisela Storz': '1188',
 'Huda Zoghbi': '1029',
 'James Manley': '1020',
 'Jessica Tyler': '1421',
 'John Huguenard': '13947',
 'John Kuriyan': '1018',
 'Jonathan Cooper': '1062',
 'Jos van der Meer': '41027',
 'Joshua Gold': '17965',
 'José Faraldo-Gómez': '13987',
 'K VijayRaghavan': '1027',
 'Karla Kirkegaard': '3645',
 'Kate Wassum': '33127',
 'Kathryn Cheah': '91149

In [10]:
weighted_keyword_valid_model = WeightedKeywordModel.from_tf_matrix(
    doc_length_weighted_editor_tf_idf,
    editor_tf_idf_vectorizer.get_feature_names(),
    choices=editor_names
)
weighted_keyword_valid_model

In [11]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [12]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [13]:
manuscript_version_for_recommendation_df = read_big_query(
    get_sql('manuscript-version-initial-submissions-for-senior-editor-recommendation.sql').format(
        **default_query_props
    )
)
manuscript_version_for_recommendation_df.head()

> ```sql
> WITH t_manuscript_version_abstract_keywords AS (
>   SELECT
>     LPAD(CAST(manuscript_abstract_keywords.manuscript_id AS STRING), 5, '0') AS manuscript_id,
>     manuscript_abstract_keywords.version_id,
>     manuscript_abstract_keywords.extracted_keywords,
>     ROW_NUMBER() OVER (
>       PARTITION BY version_id
>       ORDER BY data_hub_imported_timestamp DESC
>     ) AS version_id_row_number
>   FROM `elife-data-pipeline.de_dev.manuscript_abstract_keywords` AS manuscript_abstract_keywords
>   WHERE ARRAY_LENGTH(extracted_keywords) > 0
> ),
> 
> t_last_manuscript_version_abstract_keywords AS (
>   SELECT
>     * EXCEPT(version_id_row_number)
>   FROM t_manuscript_version_abstract_keywords
>   WHERE version_id_row_number = 1
>   ORDER BY version_id
> )
> 
> SELECT version.version_id, manuscript_version_abstract_keywords.extracted_keywords
> FROM `elife-data-pipeline.de_dev.mv_manuscript_version` AS version
> JOIN t_last_manuscript_version_abstract_keywords AS manuscript_version_abstract_keywords
>   ON manuscript_version_abstract_keywords.version_id = version.version_id
> WHERE version.overall_stage = 'Initial Submission'
>   AND ARRAY_LENGTH(version.senior_editors) = 0
>   AND TIMESTAMP_DIFF(CURRENT_TIMESTAMP, version.created_timestamp, DAY) < 365
> ```

Downloading: 100%|██████████| 135/135 [00:01<00:00, 116.43rows/s]


,version_id,extracted_keywords
0,49069/2019-06-05T12:39:51Z,"[1 expression level, accurate, accurate model,..."
1,49224/2019-06-11T08:05:08Z,"[activity, association, bacteria, charge, char..."
2,49391/2019-06-17T03:54:44Z,"[4.1 protein, associate, associated protein, b..."
3,50111/2019-07-11T05:25:04Z,"[activity, axis, basal, basal expression, basa..."
4,51945/2019-09-17T10:10:25Z,"[activation, activation signaling, adhesion, a..."


In [14]:
keyword_similarity = cosine_similarity(
    editor_tf_idf_vectorizer.transform(
        manuscript_version_for_recommendation_df
        ['extracted_keywords']
    ),
    doc_length_weighted_editor_tf_idf
)
print(keyword_similarity.max())
keyword_similarity

0.3355734086321363


array([[0.17789009, 0.09409137, 0.13246859, ..., 0.11589037, 0.11555229,
        0.12619189],
       [0.07523874, 0.06317391, 0.10638405, ..., 0.07578461, 0.1125783 ,
        0.1178674 ],
       [0.07911117, 0.05118664, 0.16084152, ..., 0.09419436, 0.18452799,
        0.11085467],
       ...,
       [0.0872426 , 0.05539434, 0.10876117, ..., 0.07425155, 0.10561886,
        0.07420016],
       [0.10689867, 0.07306304, 0.18234542, ..., 0.20835954, 0.16439594,
        0.12621397],
       [0.05188661, 0.0316176 , 0.03079543, ..., 0.02175103, 0.02470023,
        0.02204759]])

In [15]:
weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords'][:1],
).proba_matrix

matrix([[ 8.16809431,  1.00113205,  0.78019675,  2.30711144, 26.72830883,
          0.63857876, 15.92986308,  2.73732652,  1.30533635,  5.86771201,
          5.20698991,  2.66685873,  3.78807244,  0.33853684,  1.09000712,
          0.84766577,  4.81300858,  3.08706721,  9.80963766,  3.07018969,
          0.38567689,  9.19183031,  1.11259542,  0.61039891,  0.94239159,
          1.060851  ,  5.29970796,  0.53443191,  1.31372048, 11.05265012,
          1.50147106,  6.53629476,  0.7358252 ,  3.95242634,  2.90255003,
          6.895801  ,  2.69345303,  1.05095493,  1.22104209,  0.7132364 ,
         13.26133051,  5.44627183,  6.30727615,  0.97866159,  1.01662547,
         37.61324657,  1.4288973 ,  3.45346046,  2.64423708,  1.40615545,
          1.75553731,  9.66685181,  7.59101581,  0.54344678,  0.80012483,
          5.56218814,  1.21850571,  2.37949061,  2.40962799,  0.71846695,
         12.3067333 ,  0.49726166,  3.45078221,  0.68093046,  0.88666095]])

In [16]:
manuscript_matching_keywords_list = weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords']
).matching_keywords_list
pd.Series(manuscript_matching_keywords_list[:5])

0    [[(1.000710592546314, cell), (0.94548231009763...
1    [[(0.43944952441157575, mechanism), (0.3921703...
2    [[(1.000710592546314, cell), (0.49271613343116...
3    [[(1.000710592546314, cell), (0.49271613343116...
4    [[(0.9454823100976327, model), (0.492716133431...
dtype: object

In [17]:
T = TypeVar('T')


def get_recommended_editors_with_probability(
        proba_matrix: List[List[float]],
        manuscript_matching_keywords_list: List[List[Tuple[float, str]]],
        indices: List[T],
        threshold: float = 0.5) -> List[List[Tuple[float, T]]]:
    return [
        sorted([
            (
                p,
                key,
                sum(
                    s for s, _ in editor_matching_keywords
                ),
                editor_matching_keywords
            )
            for p, key, editor_matching_keywords in zip(
                row,
                indices,
                editors_matching_keywords
            ) if p >= threshold
        ], reverse=True)
        for row, editors_matching_keywords in zip(proba_matrix, manuscript_matching_keywords_list)
    ]


prediction_results_with_similarity = pd.Series(
    get_recommended_editors_with_probability(
        keyword_similarity,
        manuscript_matching_keywords_list,
        editor_names,
        threshold=0.001
    ),
    index=manuscript_version_for_recommendation_df.index
)
# print(prediction_results_with_similarity[0])
prediction_results_with_similarity[:5]

0    [(0.20284825072707918, Naama Barkai, 1.4288973...
1    [(0.16296864597421715, Philip Cole, 1.69299488...
2    [(0.2638693234976934, Dominique Soldati-Favre,...
3    [(0.2533793660157667, Maureen Murphy, 8.174770...
4    [(0.2237822437909484, Olga Boudker, 5.95258632...
dtype: object

In [18]:
prediction_results_df = pd.concat([
    manuscript_version_for_recommendation_df['version_id'],
    prediction_results_with_similarity.to_frame('prediction'),
], axis=1)
print(len(prediction_results_df))
prediction_results_df.head()

135


,version_id,prediction
0,49069/2019-06-05T12:39:51Z,"[(0.20284825072707918, Naama Barkai, 1.4288973..."
1,49224/2019-06-11T08:05:08Z,"[(0.16296864597421715, Philip Cole, 1.69299488..."
2,49391/2019-06-17T03:54:44Z,"[(0.2638693234976934, Dominique Soldati-Favre,..."
3,50111/2019-07-11T05:25:04Z,"[(0.2533793660157667, Maureen Murphy, 8.174770..."
4,51945/2019-09-17T10:10:25Z,"[(0.2237822437909484, Olga Boudker, 5.95258632..."


In [19]:
prediction_results_df['prediction'][0]

[(0.20284825072707918,
  'Naama Barkai',
  1.428897302758835,
  [(0.1738678581486904, 'cell'),
   (0.12478818046177854, 'model'),
   (0.07537932592902835, 'expression'),
   (0.07389383971319342, 'dynamic'),
   (0.06988150452514671, 'network'),
   (0.05959541072185202, 'level'),
   (0.05235176963963797, 'factor'),
   (0.05136400040115423, 'response'),
   (0.050376231162670496, 'role'),
   (0.043801325802843155, 'single'),
   (0.04338454049080308, 'molecular'),
   (0.033547953764553454, 'mathematical'),
   (0.03310176530138529, 'type'),
   (0.032620956504424065, 'cancer'),
   (0.02951571197325128, 'stress'),
   (0.028492017754017177, 'prediction'),
   (0.023835559381679725, 'concentration'),
   (0.023825936171178744, 'measurement'),
   (0.023461296800489414, 'variation'),
   (0.023405288596939095, 'disease'),
   (0.02107241042098635, 'major'),
   (0.02094640066874885, 'heterogeneity'),
   (0.01963782659388495, 'phenotypic'),
   (0.015706386465093067, 'shape'),
   (0.01530365860701376, 'v

In [20]:
prediction_results_flat_df = pd.DataFrame([
    {
        'version_id': row.version_id,
        'score': predicted_editor[0],
        'name': predicted_editor[1],
        'person_id': editor_person_id_by_name_map[predicted_editor[1]],
        'matching_keyword_score': predicted_editor[2],
        'matching_keywords': [{
            'score': keyword_score,
            'keyword': keyword
        } for keyword_score, keyword in predicted_editor[3]],
    }
    for row in prediction_results_df.itertuples()
    for predicted_editor in row.prediction
])
print(len(prediction_results_flat_df))
prediction_results_flat_df.head()

8639


,version_id,score,name,person_id,matching_keyword_score,matching_keywords
0,49069/2019-06-05T12:39:51Z,0.202848,Naama Barkai,1725,1.428897,"[{'score': 0.1738678581486904, 'keyword': 'cel..."
1,49069/2019-06-05T12:39:51Z,0.177890,Aleksandra Walczak,50904,8.168094,"[{'score': 1.000710592546314, 'keyword': 'cell..."
2,49069/2019-06-05T12:39:51Z,0.140364,Maureen Murphy,33764,5.446272,"[{'score': 0.9017922056191324, 'keyword': 'can..."
3,49069/2019-06-05T12:39:51Z,0.138349,Detlef Weigel,1030,0.338537,"[{'score': 0.029681153023506546, 'keyword': 'e..."
4,49069/2019-06-05T12:39:51Z,0.137367,Philip Cole,21020,1.755537,"[{'score': 0.2551485461419908, 'keyword': 'cel..."


In [21]:
prediction_results_flat_df['version_id'].nunique()

134

In [22]:
prediction_results_flat_df.max()

version_id                58638/2020-05-06T09:35:13Z
score                                       0.335573
name                                   Wendy Garrett
person_id                                      97155
matching_keyword_score                       103.236
dtype: object

In [23]:
print('writing to:', recommendation_output_table_name)
to_gbq(
    prediction_results_flat_df,
    recommendation_output_table_name,
    project_id=project_id,
    if_exists='replace'
)
print('done')

writing to: de_dev.data_science_editor_recommendation
done
